# Getting Started in GRASS with Python, Pixels, and Pigs

***Anna Petrasova, Vaclav Petras and Caitlin Haedrich***

*Center for Geospatial Analytics at NC State University*

In this notebook, we will introduce GRASS and then we will look at an example case study in a small watershed in Eastern North Carolina.

By the end of this notebook, you will have experience with:

*   [Starting a new GRASS project](#start)
*   [Importing data from cloud](#import)
*   [Terrain analysis](#ndvi)
*   [Computing NDVI](#ndvi)
*   [Segmenting imagery to extract waterbodies](#segmentation)
*   Common hydrology tools for [extracting streams](#streams), [computing flow paths](#drain) and [modeling innundation](#hand) using the Heigh Above Nearest Drainage (HAND) method [(Nobre et al., 2011)](https://www.sciencedirect.com/science/article/pii/S0022169411002599).

Let's start with what is [GRASS](https://grass.osgeo.org/)! Check out GRASS [documentation](https://grass.osgeo.org/grass-devel/manuals/index.html) including [GRASS tools](https://grass.osgeo.org/grass-devel/manuals/full_index.html)
and [interfaces](https://grass.osgeo.org/grass-devel/manuals/interfaces_overview.html).

---



<a name="start"></a>
### Start GRASS and Create a New Project

Now, we'll import the Python APIs for GRASS, `grass.script` and `grass.jupyter`. We'll need to ask `grass` to check it's `--config` to see where the python packages are then add them to the system path before we can import them.

In [ ]:
import sys
import subprocess

# Ask GRASS where its Python packages are to be able to run it from the notebook
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)
import grass.script as gs
import grass.jupyter as gj

With our packages imported, we can now create a new [project](https://grass.osgeo.org/grass-dev/manuals/grass_projects.html) called "workshop".

GRASS organizes geospatial work into GRASS projects and mapsets. Each project uses a single coordinate reference system (CRS). Inside a project are mapsets, which act as subprojects where data and analyses are organized. Mapsets can represent different research tasks, study areas, or the work of individual collaborators.

![Projects and mapsets](https://grass.osgeo.org/grass-devel/manuals/project.png)

When we create a new project, we can set the CRS from a georeferenced file (such as a Geotiff) or an EPSG string. Here, we use [EPSG 3358](https://epsg.io/3358), a projection for NC in meters.

In [ ]:
gs.create_project("workshop", crs="EPSG:3358")

We start a GRASS session in our new project.

In [ ]:
gj.init("workshop");

In this workshop we will be using new GRASS Python API using GRASS preview version.

In [ ]:
from grass.tools import Tools

tools = Tools()
tools.g_search_modules(keyword="hydrology")



---


<a name="import"></a>
## Import Data

Import our area of interest (AOI) from a GeoJSON file.

In [ ]:
tools.v_import(input="AOI.geojson", output="AOI")

In [ ]:
aoi_map = gj.InteractiveMap()
aoi_map.add_vector("AOI")
aoi_map.show()
    

There are lots of ways to set the computational region (see [g.region](https://grass.osgeo.org/grass-devel/manuals/g.region.html) documentation). We will start by setting the region to the extent of our AOI.

In [ ]:
tools.g_region(vector="AOI", res=1)
gs.region()

We're going to import a digital elevation model (DEM), we will use a GRASS addon [r.in.usgs](https://grass.osgeo.org/grass-devel/manuals/addons/r.in.usgs.html), which uses [TNM Access](https://apps.nationalmap.gov/tnmaccess/) REST API to access USGS data. First install the addon:

In [ ]:
tools.g_extension(extension="r.in.usgs")

Download and reproject a 1/9 arc-second DEM (approx 3-m resolution):

In [ ]:
tools.r_in_usgs(product="ned", ned_dataset="ned19sec", flags="i")

In [ ]:
tools.r_in_usgs(product="ned", ned_dataset="ned19sec", output_name="elevation", verbose=True)

<details>

<summary>Alternative Import Method</summary>

### Download with wget and import with `r.import`

First, download and unzip with bash.

```bash
%%bash
wget https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/19/IMG/ned19_n35x00_w078x00_nc_statewide_2003.zip
unzip ned19_n35x00_w078x00_nc_statewide_2003.zip
```

We'll import our elevation model using [`r.import`](https://grass.osgeo.org/grass-devel/manuals/r.import.html) and create a raster layer called "elevation". The `r.import` tool will reproject the data to the project CRS (thereby avoiding any future CRS mismatches - nice!). We also set it to only import the area within the computational region and to resample it using bilinear interpolation to the resolution of the computational region.

```python
gs.run_command("r.import", input="ned19_n35x00_w078x00_nc_statewide_2003.img", output="elevation", resample="bilinear", extent="region")
```

</details>

Let's display our elevation layer using the `grass.jupyter.Map()` class. The `Map()` class creates and displays GRASS maps as PNG images. `Map()` accepts any [GRASS display tools](https://grass.osgeo.org/grass-devel/manuals/display.html) as a method by replacing the "." with "_" in the module name.
To display the image, we call the show() method. You can also save the image with the save() method.

In [ ]:
map = gj.Map()
map.d_rast(map="elevation")
map.show()

Now set the computational region to match the elevation layer.

In [ ]:
tools.g_region(raster="elevation")



---


## Terrain analysis

Now for our first terrain analysis example. In the following cell, we use [r.relief](https://grass.osgeo.org/grass-devel/manuals/r.relief.html) to compute a shaded relief map:

In [ ]:
gs.run_command("r.relief", input="elevation", output="relief")

map = gj.Map()
map.d_shade(color="elevation", shade="relief", brighten=50)
map.d_barscale()
map.show()

Let's look at one example of a terrain analysis tool available in GRASS. The [r.geomorphon](https://grass.osgeo.org/grass-devel/manuals/r.geomorphon.html) tool classifies terrain into it's different landforms (i.e. peaks, valleys, ridges,...).

<img src="https://grass.osgeo.org/grass-stable/manuals/legend.png" />

In [ ]:
tools.r_geomorphon(elevation="elevation", forms="landforms", skip=11, search=25)

map = gj.Map()
map.d_shade(color="landforms", shade="relief", brighten=50)
map.d_legend(
    raster="landforms",
    at=(0, 40, 1, 5),
    title="Landforms",
    border_color="none",
    flags="bt"
)
map.d_barscale(at=(73, 5))
map.show()

Let's look into the effect of the parameters of the r.geomorphon tool. We will zoom in on a smaller area in the middle using [Region Manager](https://grass.osgeo.org/grass-devel/manuals/libpython/grass.script.html#grass.script.RegionManager) that restricts the computation to a subregion.

Then we display the landforms at different scales as an animation using [SeriesMap](https://grass.osgeo.org/grass85/manuals/jupyter_intro.html#series-map):

In [ ]:
with gs.RegionManager(grow=-500):
    for i, skip, search in ((1, 7, 15), (2, 15, 25), (3, 25, 50)):
        tools.r_geomorphon(elevation="elevation", forms=f"landforms_{i}", skip=skip, search=search)
    
series_map = gj.SeriesMap()
series_map.add_rasters([f"landforms_{i + 1}" for i in range(3)])
series_map.d_legend(
    raster="landforms",
    at=(0, 40, 1, 5),
    title="Landforms",
    border_color="none",
    flags="bt"
)
series_map.show()


---

## Case Study: Swine lagoons



North Carolina is one of the top pork producing states in the US, surpassed only by Minnesota and Iowa. The large industry in North Carolina consists of hundreds of large-scale farms raise pigs, processing facilities, trucks that transport the animals and fields that grow the grains for feed. Raising over 8 million pigs in a concentrated area creates one big issue: waste.

<img src="https://raw.githubusercontent.com/chaedri/chaedri.github.io/refs/heads/master/images/CAFOs.png" />

The waste is typically stored in large retention ponds referred to as *lagoons*. The waste anaerobically digests and then is spread on the nearby fields as fertilizer. However, during catastorphic flooding events such as [Hurricane Florence in 2018](https://www.npr.org/2018/09/22/650698240/hurricane-s-aftermath-floods-hog-lagoons-in-north-carolina), the flood waters can overtop the sides of the lagoon introducing the waste to the surrounding environment.

<img src="https://modernfarmer.com/wp-content/uploads/2022/02/16442235689_6f9667cc05_k-768x489.jpg" />

Using the near infrared band, we will use an image segmentation tool to isolate the lagoons. Then, we'll use some of the hydrology tools to extract the streams and simulated an innudation flood around the streams to see which lagoons will flood.



### Automated Lagoon Detection

#### Get Imagery
First, let's get some aerial imagery of the study area by importing data through a WMS service.

In [ ]:
tools.r_in_wms(url="https://imagery.nationalmap.gov/arcgis/services/USGSNAIPPlus/ImageServer/WMSServer", out="ortho", layer="USGSNAIPPlus")

map = gj.InteractiveMap()
map.add_raster("ortho", opacity=0.7)
map.show()

In [ ]:
tools.r_rgb(input="ortho", red="red", green="green", blue="blue")

map = gj.SeriesMap()
map.add_rasters(["red", "green", "blue"])
map.show()

In addition to RGB imagery, we'll get false color imagery that uses the Near Infra-Red (NIR) band.

In [ ]:
tools.r_in_wms(url="https://imagery.nationalmap.gov/arcgis/services/USGSNAIPPlus/ImageServer/WMSServer", out="naip_false", layer="USGSNAIPPlus:FalseColorComposite")

map = gj.InteractiveMap()
map.add_raster("naip_false", opacity=0.7)
map.show()

<details>

<summary>Alternative Import Method</summary>

```bash
%%bash
r.unpack ortho.pack
r.unpack naip_false.pack
```

</details>

The NAIP false color image is a composite of the NIR, red and green bands. We use the [r.rgb](https://grass.osgeo.org/grass-devel/manuals/r.rgb.html) tool to separate the bands into separate layers.

In [ ]:
tools.r_rgb(input="naip_false", red="nir", green="red", blue="green")

map = gj.SeriesMap()
map.add_rasters(["nir", "red", "green"])
map.show()

<a name="ndvi"></a>

Since we have the NIR band, we can now compute NDVI:

$$
NDVI = \frac{NIR - Red}{NIR + Red}
$$

In [ ]:
tools.r_mapcalc(expression="ndvi = ((float(nir) - float(red)) / (float(nir) + float(red)))")

# Use built-in NDVI color pallete
tools.r_colors(map="ndvi", color="ndvi")

map = gj.InteractiveMap()
map.add_raster("ndvi", opacity=0.5)
map.show()

And NDWI:


$$
NDWI = \frac{Green - NIR}{Green + NIR}
$$

In [ ]:
tools.r_mapcalc(expression="ndwi = ((float(green) - float(nir)) / (float(nir) + float(green)))")

# Use built-in NDWI color pallete
tools.r_colors(map="ndwi", color="ndwi")

map = gj.InteractiveMap()
map.add_raster("ndwi", opacity=0.5)
map.show()

<a name="segmentation"></a>
#### Lagoon Extraction with Image Segmentation

Looking at the NIR band, it's clear that the lagoons and water bodies have the lowest values. We will isolate them by segmenting the image using all 4 bands ([i.segment](https://grass.osgeo.org/grass-devel/manuals/i.segment.html)) then using zonal statistics with the NIR band ([r.stats.zonal](https://grass.osgeo.org/grass-devel/manuals/r.stats.zonal.html)) to find the patches with the lowest NIR values. Using a threshold, we separate the lagoons from the other patches.

First, we segment the image.

In [ ]:
tools.i_segment(group=["nir", "red", "green", "blue"], output="segments", threshold=0.5, minsize=300)

map = gj.Map()
map.d_rast(map="segments")
map.show()

Use [r.stats.zonal](https://grass.osgeo.org/grass-devel/manuals/r.stats.zonal.html) to compute the average NIR value for each patch.

In [ ]:
tools.r_stats_zonal(base="segments", cover="nir", method="average", output="segments_nir")

map = gj.InteractiveMap(width=900, height=500)
map.add_raster("segments_nir")
map.show()

That leaves us with a lot of false detections, mostly places along forest edge:

In [ ]:
map = gj.Map(width=900, height=500)
map.d_rast(map="segments_nir", values="0-100")
map.d_legend(raster="segments_nir", flags="dt", label_step=10)
map.show()

We will use a threshold of 100 to separate the lagoons from the other patches and then select only patches with higher compactness to filter out the false patches representing shadows along forest edge.

In [ ]:
tools.r_mapcalc(expression="segments_100 = if(segments_nir < 100, segments, null())")

Convert to vector areas and compute compactness in a new attribute column.

In [ ]:
tools.r_to_vect(input="segments_100", output="segments_100", type="area")
tools.v_to_db(map="segments_100", option="compact", column="compactness")

Let's look at the histogram of compactness values.

In [ ]:
import pandas as pd
df = pd.DataFrame(tools.v_db_select(map="segments_100", column="compactness", format="json")["records"])
df

In [ ]:
df.hist(bins=30)

Based on the histogram, we will use 2.5 as a threshold to filter out more false detections.

In [ ]:
tools.v_extract(input="segments_100", output="lagoons", type="area", where="compactness < 2.5")

map = gj.InteractiveMap()
map.add_raster("nir")
map.add_vector("lagoons", opacity=0.7)
map.show()

Finally, we'll place a vector point in the centroid of each lagoon. To do that,
convert the areas in points by extracting the area centroids with
[v.type](https://grass.osgeo.org/grass-devel/manuals/v.type.html).

In [ ]:
tools.v_type(input="lagoons", output="lagoon_points", from_type="centroid", to_type="point")

map = gj.Map()
map.d_rast(map="ortho")
map.d_vect(map="lagoon_points", size=15, icon="basic/point")
map.show()

Tada! We have identified all the lagoons in the study area.


### Lagoon Flood Risk

Let's use these lagoon points to answer 4 questions:
1. If the lagoons overflowed, what path would the waste travel to the nearest waterway?
2. If the stream water level rose 1 meter, would any of the lagoon be breached?
3. What is the upstream contributing area for a hypothetical sample point?
4. What are the overland flow dynamics during a heavy rainstorm in the upstream contributing area?

<a name="drain"></a>
__Question 1:__ *If the lagoons overflowed or were breached, what path would the waste travel to the nearest waterway?*

(This does happen and has serious consequences:  [news article](https://www.newsobserver.com/news/state/north-carolina/article264779224.html)).

The [r.watershed](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html) tool is a popular and powerful tool for hydrology. Check out all of the outputs it can compute in the [manual page](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html). Here we'll use it to compute the flow accumulation (how many cells are upstream of a given cell) and drainage direction (what direction a particle would flow from each cell). By default the tool uses multiple flow direction algorithm, which works better on a flat landscape. We don't need to fill sinks, because r.watershed uses least-cost path approach for routing through depressions. Then, we'll use [r.path](https://grass.osgeo.org/grass-devel/manuals/r.path.html) to trace the route of the waste being transported downhill from the lagoon.

In [ ]:
tools.r_watershed(elevation="elevation", accumulation="accumulation", drainage="drainage")

In [ ]:
map = gj.InteractiveMap()
map.add_raster("accumulation", opacity=0.5)
map.show()

In [ ]:
tools.r_path(input="drainage", vector_path="drain", start_points="lagoon_points")

Let's see what is the landcover the water from lagoons would flow through:

In [ ]:
map = gj.Map()
map.d_shade(color="ortho", shade="relief", brighten=50)
map.d_vect(map="drain", width=1, color="blue")
map.show()

<a name="hand"></a>
__Question 2:__ *If the stream water level rose 1 meter, would any of the lagoon be breached?*

To answer this question, we'll use the HAND (height above nearest drainage) method to model the flood [(Nobre et al., 2011)](https://www.sciencedirect.com/science/article/pii/S0022169411002599).

First, we'll add the two extensions we need for this workflow.

In [ ]:
tools.g_extension(extension="r.lake.series")
tools.g_extension(extension="r.stream.distance")

<a name="streams"></a>
Use the elevation and the flow accumulation raster we computed with `r.watershed` to extract the streams and vectors. The threshold is the minimum flow accumulation for a cell to be part of the stream network.

In [ ]:
tools.r_stream_extract(elevation="elevation", accumulation="accumulation",
                       stream_raster="streams", stream_vector="streams", direction="direction", threshold=100000)
map = gj.Map()
map.d_shade(color="elevation", shade="relief", brighten=50)
map.d_vect(map="streams", width=1, color="blue", layer=1)
map.d_vect(map="lagoons", fill_color="none")
map.show()

The [r.lake](https://grass.osgeo.org/grass-devel/manuals/r.lake.html) tool is a "bathtub" model for flooding.

In [ ]:
tools.r_lake(elevation="elevation", water_level=25, lake="flood", seed="streams")
map = gj.Map()
map.d_shade(color="flood", shade="relief", brighten=50)
map.d_vect(map="streams", width=1, color="blue", layer=1)
map.d_vect(map="lagoons", fill_color="none")
map.show()

As we can see above, `r.lake` is filling the DEM from the lowest point on the the streams network. But what if we wanted to flood each section of the stream by a meter? For this, we can use a height above nearest drainage (HAND) raster instead of the elevation raster for the "elevation" parameter.
HAND represents the vertical distance between any point on a landscape and the nearest stream or drainage channel.

We can create the HAND raster using [r.stream.distance](https://grass.osgeo.org/grass-devel/manuals/addons/r.stream.distance.html).

In [ ]:
tools.r_stream_distance(stream_rast="streams", direction="direction", elevation="elevation", method="downstream", difference="HAND")
map = gj.Map()
map.d_shade(color="HAND", shade="relief", brighten=50)
map.d_vect(map="streams", width=1, color="blue", layer=1)
map.d_vect(map="lagoons", fill_color="none")
map.show()

In [ ]:
tools.r_lake(elevation="HAND", water_level=1, lake="flood", seed="streams")
map = gj.Map()
map.d_shade(color="flood", shade="relief", brighten=50)
map.d_vect(map="streams", width=1, color="blue", layer=1)
map.d_vect(map="lagoons", fill_color="none")
map.show()

To create a timeseries of the innundation, we can use [r.lake.series](https://grass.osgeo.org/grass-devel/manuals/addons/r.lake.series.html).

In [ ]:
tools.r_lake_series(elevation="HAND", seed_raster="streams", start_water_level=0, end_water_level=1, water_level_step=0.1, output="inundation")


In [ ]:
map = gj.TimeSeriesMap()
map.d_rast(map="relief")
map.d_vect(map="lagoons", fill_color="none")
map.add_raster_series("inundation")
map.show()

__Question 3:__ *What is the upstream contributing area for a hypothetical sample point?*

To do this, we will extract a coordinate from a section of stream and then use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) with the drainage direction raster to compute the upstream contribute area.

In [ ]:
tools.v_extract(input="streams", output="stream_points", type="point")

In [ ]:
map = gj.Map()
map.d_rast(map="relief")
map.d_vect(map="streams", color="blue")
map.d_vect(map="stream_points", display="cat", label_color="white", label_size=10)
map.show()

Let's use point with category 9. Vector attributes are stored in a SQL database inside the project. We use [v.to.db](https://grass.osgeo.org/grass-devel/manuals/v.to.db.html) to add the feature coordinates to the attribute table, then [v.db.select](https://grass.osgeo.org/grass-devel/manuals/v.db.select.html) to select the category and coordinates and show them as a Pandas dataframe.

In [ ]:
import pandas as pd

gs.run_command("v.to.db", map="stream_points", option="coor", type="point", columns="x,y")
df = pd.DataFrame(gs.parse_command("v.db.select", map="stream_points", columns="cat,x,y", format="json")["records"])
df

Finally, use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) to compute the upstream contributing area.

In [ ]:
gs.run_command("r.water.outlet", input="direction", output="basin", coordinates=[df.loc[8, 'x'], df.loc[8, 'y']])
map = gj.Map()
map.d_shade(color="basin", shade="relief", brighten=50)
map.show()

__Question 4:__ *What are the overland flow dynamics during a heavy rainstorm in the identified basin?*

We're going to use [r.sim.water](https://grass.osgeo.org/grass-devel/manuals/r.sim.water.html) to model overland flow. The `r.sim.water` tool is the GRASS implementation of the SIMWE model ([Mitas and Mitasova, 1998](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/97WR03347)), a monte carlo path-tracing approach to solving the St. Venant equations for overland flow.

We will restrict the computational region to the basin of interest and lower the resolution to make computation faster.
Additinally, we will apply a mask over the areas outside `basin`. Now, only areas inside `basin` will be displayed or used in any computation. Unlike computational region, [r.mask](https://grass.osgeo.org/grass-devel/manuals/r.mask.html) can have boundaries that are not rectangular.
This is done using RegionManager and MaskManager objects.

Run `r.sim.water` after computing the x and y direction derivatives. We'll run a 30-minute rainstorm using the default rainfall rate of 50 mm/hr. The output will be a series a map showing water depth at each minute.

In [ ]:
with gs.RegionManager(zoom="basin", res=6):
    with gs.MaskManager(mask_name="basin"):
        gs.run_command('r.slope.aspect', elevation="elevation", dx="dx", dy="dy")
        gs.run_command('r.sim.water', elevation="elevation", dx="dx", dy="dy", depth="depth", flags="t", niterations=30)

Finally, we'll create a temporal dataset and register the output depth maps. GRASS has [a library of tools](https://grass.osgeo.org/grass-devel/manuals/temporalintro.html) for temporal analyses but here, we will just create an animation of the timeseries.

In [ ]:
# Create a time series
gs.run_command("t.create",
               output="depth",
               temporaltype="relative",
               title="Overland flow depth",
               description="Overland flow depth")

# Register the time series
maps = gs.list_strings(type="raster", pattern="depth*")
gs.run_command("t.register", input="depth", maps=maps)

In [ ]:
flow_map = gj.TimeSeriesMap()
flow_map.add_raster_series("depth", fill_gaps=True)
flow_map.show()